***
__ClassFractions__ fractions (frequency) image for class values in as  ImageCollection of classifications
***

In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils
import geeproduct
import geemask

In [ ]:
eepoint            = geeutils.half31UESpoint #geeutils.tennvenlopoint #geeutils.brusselspoint       #geeutils.half31UESpoint #
eedatefrom         = geeutils.half31UESday #ee.Date('2018-01-01') 
#eedatefrom         = eedatefrom.advance(2, 'day')
eedatetill         = eedatefrom.advance(1, 'year')
eeroi              = geeutils.squareareaboundsroi(eepoint, 100/2*20)
verbose            = False

rgbimage           = geeproduct.GEECol_s2rgb().collect(eeroi, eedatefrom, eedatetill, verbose=verbose).first()

sclimagecollection = geeproduct.GEECol_s2scl().collect(eeroi, eedatefrom, eedatetill, verbose=verbose)
sclimage           = sclimagecollection.first()

classfractionsmaker = geemask.ClassFractions([4,5,6], [8,9,10]) #geemask.ClassFractions([8,9,10])
fractionsimage      = classfractionsmaker.makefractions(sclimagecollection)
print (geeutils.szprojectioninfo(fractionsimage))
fractionsimage      = fractionsimage.setDefaultProjection(sclimagecollection.first().projection())
print (geeutils.szprojectioninfo(fractionsimage))
print (geeutils.szestimatevaluesinfo(fractionsimage.clip(eeroi))) # clip; otherwise unbounded
xin = fractionsimage.clip(eeroi).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xax = fractionsimage.clip(eeroi).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()
print (xin, xax)


map = geemap.Map(height='600px')
map.centerObject(eepoint, 12)
map.addLayer(rgbimage,                                   {'min': 1,  'max':255},                 'rgb')
map.addLayer(sclimage,                                   geeutils.s2sclvisParams,                'scl')

map.addLayer(fractionsimage,                             {'min':xin,  'max':xax},                'fct')                        
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 1), {'palette':'#ff0000'},                  'roi')
map

In [ ]:
eepoint       = geeutils.antwerppoint
eedatefrom    = ee.Date('2018-01-01') 
eedatetill    = eedatefrom.advance(1, 'year')
eeroi         = geeutils.squareareaboundsroi(eepoint, 10000)
verbose       = False

eeimagecollection   = geeproduct.GEECol_s2scl().collect(eeroi, eedatefrom, eedatetill, verbose=verbose)
classfractionsmaker = geemask.ClassFractions([8,9,10])
cloudfractionsimage = classfractionsmaker.makefractions(eeimagecollection)
print (geeutils.szprojectioninfo(cloudfractionsimage))

cloudfractionsimage = cloudfractionsimage.setDefaultProjection(eeimagecollection.first().projection())
print (geeutils.szprojectioninfo(cloudfractionsimage))
print (geeutils.szestimatevaluesinfo(cloudfractionsimage.clip(eeroi))) # clip; otherwise unbounded
xin = cloudfractionsimage.clip(eeroi).reduceRegion(ee.Reducer.min()).values().get(0).getInfo()
xax = cloudfractionsimage.clip(eeroi).reduceRegion(ee.Reducer.max()).values().get(0).getInfo()

map = geemap.Map(height='600px')
map.centerObject(eepoint, 11)
map.addLayer(cloudfractionsimage,                              {'min':xin,  'max':xax}) #, 'opacity':0.95},   'clouds')                        
map.addLayer(geeutils.outlinegeometryimage(eeroi, 0, 1),       {'palette':'#ff0000'})
map
